# Decomposing Qiskit circuits

In this notebook, we show how Qiskit circuits can be converted into Perceval circuits. We also show the equivalence between the two circuits.

In [1]:
import perceval as pcvl
import perceval.lib.phys as phys
from perceval.converters import QiskitConverter

from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

## Qiskit CNOT gate

First, we define our Qiskit CNOT circuit with 3 qubits. We put a Hadamar gate at the beginning on qubit 0 to have a state superposition and we add two CNOT gates using the qubit 0 as a control and qubits 1 and 2 as targets.

In [2]:
# Create a Quantum Circuit acting on the q register
circuit = QuantumCircuit(3, 3)

# Add a H gate on qubit 0
circuit.h(0)

# Add CX (CNOT) gates on control qubit 0 and target qubits 1 and 2
circuit.cx(0, 1)
circuit.cx(0, 2)

# Draw the circuit
circuit.draw()

┌───┐          
q_0: ┤ H ├──■────■──
     └───┘┌─┴─┐  │  
q_1: ─────┤ X ├──┼──
          └───┘┌─┴─┐
q_2: ──────────┤ X ├
               └───┘
c: 3/═══════════════

We can show the final state when starting from a ground state.

In [3]:
# Set the initial state of the simulator to the ground state using from_int
state = Statevector.from_int(0, 2**3)

# Evolve the state by the quantum circuit
state = state.evolve(circuit)

#draw using latex
state.draw('latex')

<IPython.core.display.Latex object>

## Conversion to Perceval

With the use of the `QiskitConverter`, we can transform the Qiskit circuit into a Perceval circuit using two times more modes than qubits encoding qubits, plus modes that are necessary to make the cnot work. The first six modes corresponds to the three qubits (see the Spatial Modes encoding paragraph in the Basics section of the documentation) and the six other modes are the necessary modes to make the cnot work (note that they are all heralded to 0 or 1 photons in the output, meaning we only consider output states having the given number of photons in these modes).

In [4]:
qiskit_converter = QiskitConverter(phys)
quantum_processor = qiskit_converter.convert(circuit)
pcvl.pdisplay(quantum_processor, recursive=True)

With this converted circuit, we can now check that the resulting state is the same as before the conversion.

In [5]:
simulator_backend = pcvl.BackendFactory().get_backend("Naive")

_, output_distribution = quantum_processor.run(simulator_backend)
pcvl.pdisplay(output_distribution)

state,probability
"|1,0,0,1,1,0,0,1,1,0,0,0>",0.497767
"|1,1,0,1,0,0,0,1,1,0,0,0>",0.155552
"|1,1,1,0,0,0,0,1,1,0,0,0>",0.148261
"|1,0,1,0,1,0,0,1,1,0,0,0>",0.124442
"|0,1,1,0,1,0,0,1,1,0,0,0>",0.031597
"|1,2,0,0,0,0,0,1,1,0,0,0>",0.016527
"|0,1,0,1,1,0,0,1,1,0,0,0>",0.007778
"|0,0,0,1,2,0,0,1,1,0,0,0>",0.004861
"|1,0,0,2,0,0,0,1,1,0,0,0>",0.003889
"|0,2,0,1,0,0,0,1,1,0,0,0>",0.003889


Note that we can replace the permutations in the above Perceval circuit using a real photonic chip with a general interferometer decomposition.

In [9]:
u = quantum_processor.circuit.compute_unitary(use_symbolic=False)

In [10]:
ub = (pcvl.Circuit(2)
      // phys.BS(theta=pcvl.Parameter("theta"))
      // (0, phys.PS(phi=pcvl.Parameter("φ_a"))))

pc_norm = pcvl.Circuit.decomposition(u, ub, shape="triangle")
pcvl.pdisplay(pc_norm, compact=True, render_size=0.8)
